## Capacitated Plant Location

Optimize the global supply chain network to meet regional product demand at the lowest possible cost.

Decision Variables:

- **Production Quantity $x_{ij}$:** This represents the amount of product produced at a specific location $i$ and shipped to destination $j$. This variable will be adjusted to meet regional demands efficiently:

- **Plant Operation Status $y_{is}$:** This binary variable indicates whether a plant at location $i$ with a certain capacity ($s$, which can be either low or high) is operational ($y = 1$) or closed ($y = 0$).

- **Plant Capacity $s$:** This variable denotes the size of the production facilities, categorized as low or high capacity.

Constraints:

- **Production Capacity:** Each plant's production cannot exceed its capacity, whether it's a low or high capacity plant.

- **Demand Fulfillment:** The total production and shipping from all plants must meet or exceed the regional demand for the product.

- **Plant Operation:** A plant can only produce if it is operational (i.e., y = 1 for that plant).

Objective:

The objective is to minimize the total cost $z$:

$$
z = \sum^{n}_{i=1} \sum^{s}_{k=1} f_{isk}y_{isk} + \sum^{n}_{i=1} \sum^{m}_{j=1} c_{ij} x_{ij}
$$

where:
- $n$ is the number of production facilities
- $m$ is the number of markets or regional demands
- $s$ is the number of capacity sizes
- $f_{isk}$ is the fixed cost of keeping plant $i$ of capacity size $k$ open
- $c_{ij}$ is the cost of producing and shipping from plant $i$ to region $j$
- $x_{ij}$ is the production quantity from plant $i$ to region $j$
- $y_{isk}$ is the binary variable indicating whether a plant $i$ with capacity size $k$ is operational
- $\mathbf{P}_{isk}$ is the capacity of plant $i$ with capacity size $k$
- $\mathbf{D}_{j}$ is the demand in region $j$.

We aim to find the most cost-effective combination of production quantities, plant operational statuses, and plant capacities that satisfies the regional demand while minimizing the total cost:

## Solution

\begin{align*}
\text{minimize}     \quad   &   z = \sum^{n}_{i=1} \sum^{s}_{k=1} f_{isk}y_{isk} + \sum^{n}_{i=1} \sum^{m}_{j=1} c_{ij} x_{ij} \\
\text{subject to}   \quad   &   \sum^{m}_{j=1} x_{ij} \le \sum^{s}_{k=1} y_{isk} \times P_{isk} \quad \forall i \in \{1, \dots, n\} \\
                    \quad   &   \sum^{n}_{i=1} x_{ij} \ge D_{j} \quad \forall j \in \{1, \dots, m\} \\
                    \quad   &   x_{ij} \ge 0 \quad \forall i \in \{1, \dots, n\}, \forall j \in \{1, \dots, m\} \\
                    \quad   &   y_{isk} \in \{0, 1\} \quad \forall i \in \{1, \dots, n\}, \forall k \in \{1, \dots, s\}
\end{align*}

In [185]:
import pandas as pd
from pprint import pprint

demand = pd.read_csv("demand.csv", index_col=0)
cap = pd.read_csv("cap.csv", index_col=0)
fix_cost = pd.read_csv("fix_cost.csv", index_col=0)
var_cost = pd.read_csv("var_cost.csv", index_col=0)

dfs = [demand, cap, fix_cost, var_cost]

names = ["DEMAND", "CAPACITY", "FIXED_COSTS", "VARIABLE_COSTS"]

for i, df in enumerate(dfs):
    df.index.name = names[i]
    print(df)

         Demand
DEMAND         
USA      2719.6
Germany    84.1
Japan    1676.8
Brazil    145.4
India     156.4
          Low_Cap  High_Cap
CAPACITY                   
USA           500      1500
Germany       500      1500
Japan         500      1500
Brazil        500      1500
India         500      1500
             Low_Cap  High_Cap
FIXED_COSTS                   
USA             6500      9500
Germany         4980      7270
Japan           6230      9100
Brazil          3230      4730
India           2110      3080
                USA  Germany  Japan  Brazil  India
VARIABLE_COSTS                                    
USA               6       13     20      12     22
Germany          13        6     14      14     13
Japan            20       14      3      21     10
Brazil           12       14     21       8     23
India            17       13      9      21      8


In [240]:
from pulp import *

locs = demand.index.to_list()
size = cap.columns.to_list()
n = len(locs)

problem = LpProblem("minimize_costs", LpMinimize)

x = LpVariable.dicts("prod", ((i, j) for i in locs for j in locs), lowBound=0, cat="Continuous")
y = LpVariable.dicts("plant", ((i, s) for i in locs for s in size), cat=LpBinary)

fy = [fix_cost.loc[i, s] * y[i, s] for s in size for i in locs]
cx = [var_cost.loc[i, j] * x[i, j] for i in locs for j in locs]

problem += lpSum(fy) + lpSum(cx)

for i in locs:
    problem += lpSum([x[i, j] for j in locs]) <= lpSum([y[i, s] * cap.loc[i, s] for s in size])

for j in locs:
    problem += lpSum([x[i, j] for i in locs]) == demand.loc[j, "Demand"]

problem.solve()

problem.status

1

In [241]:
production = pd.DataFrame()
plant_status = pd.DataFrame()

for loc1, loc2 in x.keys():
    production.loc[loc1, loc2] = x[loc1, loc2].varValue

for l, s in y.keys():
    plant_status.loc[l, s] = y[l, s].varValue

production.index.name = "PRODUCTION"
plant_status.index.name = "PLANT_STATUS"

for df in [production, plant_status]:
    print(df, "\n")

print(f"Total Cost: ${problem.objective.value():,.2f}")

               USA  Germany   Japan  Brazil  India
PRODUCTION                                        
USA         1500.0      0.0     0.0     0.0    0.0
Germany        0.0      0.0     0.0     0.0    0.0
Japan          0.0      0.0  1500.0     0.0    0.0
Brazil      1219.6      0.0     0.0   145.4    0.0
India          0.0     84.1   176.8     0.0  156.4 

              Low_Cap  High_Cap
PLANT_STATUS                   
USA               0.0       1.0
Germany           0.0       0.0
Japan             0.0       1.0
Brazil            0.0       1.0
India             1.0       0.0 

Total Cost: $58,674.10
